In [1]:
import pandas as pd
import sqlite3

from scipy.spatial.distance import cosine

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../database/gamender.db")
df = pd.read_sql_query("SELECT handler_id, game_id  FROM handlers_games", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

   handler_id  game_id
0           1        1
1           1        2
2           1        4
3           2        1
4           2       18


iloc

In [2]:
df.set_index('handler_id')

,game_id
handler_id,
1,1
1,2
1,4
2,1
2,18
...,...
1050112070,18
1220468585824505856,18
3341912236,18


In [3]:
dfgrp = df.groupby(['handler_id'])

In [4]:
dfgrp.head()

,handler_id,game_id
0,1,1
1,1,2
2,1,4
3,2,1
4,2,18
...,...,...
14129,1050112070,18
14130,1220468585824505856,18
14131,3341912236,18
14132,1366267481288019975,18


In [5]:
df.reset_index(inplace=True)

In [6]:
gb = df.groupby(['handler_id'])
result = gb['game_id'].unique()

In [8]:
result

handler_id
1                           [1, 2, 4]
2                      [1, 18, 11, 2]
3                         [3, 17, 15]
22833                            [11]
27603                            [16]
                            ...      
1490207262798979072              [10]
1490208607098519552              [10]
1490210717936590850              [10]
1490229197490556930               [1]
1490239532654026757              [12]
Name: game_id, Length: 13734, dtype: object

In [9]:
resultframe = result.to_frame()

In [10]:
def sublist_uniques(data, sublist):
    categories = set()
    for d, t in data.iterrows():
        try:
            for j in t[sublist]:
                categories.add(j)
        except:
            pass
    return list(categories)

In [11]:
import numpy as np


def sublists_to_dummies(f, sublist, index_key=None):
    categories = sublist_uniques(f, sublist)
    frame = pd.DataFrame(columns=categories)
    for d, i in f.iterrows():
        if type(i[sublist]) == list or np.array:
            try:
                if index_key is not None:
                    key = i[index_key]
                    f = np.zeros(len(categories))
                    for j in i[sublist]:
                        f[categories.index(j)] = 1
                    if key in frame.index:
                        for j in i[sublist]:
                            frame.loc[key][j] += 1
                    else:
                        frame.loc[key] = f
                else:
                    f = np.zeros(len(categories))
                    for j in i[sublist]:
                        f[categories.index(j)] = 1
                    frame.loc[d] = f
            except:
                pass

    return frame

In [12]:
sparseframe = sublists_to_dummies(resultframe, 'game_id')

In [13]:
sparseframe

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
22833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490207262798979072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1490208607098519552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1490210717936590850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1490229197490556930,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
sparseframe.index.name = 'handler_id'

# Item Based Collaborative Filtering

In [15]:
data = sparseframe.reset_index()

## Drop handler

In [16]:
data = data.drop('handler_id', 1)

C:\Users\beel\AppData\Local\Temp/ipykernel_16924/4225273426.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('handler_id', 1)


In [17]:
data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13732,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
data_ibs = pd.DataFrame(index=data.columns, columns=data.columns)

In [19]:
for i in range(0, len(data_ibs.columns)):
    # Loop through the columns for each column
    for j in range(0, len(data_ibs.columns)):
        # Fill in placeholder with cosine similarities
        data_ibs.iloc[i, j] = 1 - cosine(data.iloc[:, i], data.iloc[:, j])

In [20]:
data_ibs

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,1,0.015397,0.00872,0.001683,0.001899,0.0,0.010306,0.005675,0.010652,0.017379,0.030257,0.003866,0.008783,0.012147,0.005268,0.010355,0.00829,0.004663
2,0.015397,1,0.00872,0.003787,0.002136,0.0,0.017391,0.004256,0.00856,0.019552,0.023827,0.001087,0.011116,0.003154,0.003556,0.016308,0.01088,0.010492
3,0.00872,0.00872,1,0.00715,0.00605,0.002612,0.009849,0.005022,0.009695,0.004342,0.012851,0.002052,0.004663,0.002977,0.003356,0.012095,0.013204,0.002971
4,0.001683,0.003787,0.00715,1,0.010511,0.0,0.007606,0.0,0.002808,0.0,0.020468,0.005349,0.0,0.005172,0.0,0.005731,0.007647,0.0
5,0.001899,0.002136,0.00605,0.010511,1,0.0,0.0,0.0,0.003167,0.002128,0.002099,0.0,0.002285,0.001945,0.002193,0.002155,0.005751,0.001941
6,0.0,0.0,0.002612,0.0,0.0,1,0.0,0.0,0.0,0.0,0.00136,0.0,0.00444,0.0,0.0,0.001396,0.0,0.0
7,0.010306,0.017391,0.009849,0.007606,0.0,0.0,1,0.004273,0.012032,0.009238,0.025063,0.005458,0.014881,0.003166,0.00119,0.02807,0.006242,0.005267
8,0.005675,0.004256,0.005022,0.0,0.0,0.0,0.004273,1,0.00631,0.002119,0.007318,0.0,0.004552,0.000969,0.001092,0.004293,0.002864,0.000967
9,0.010652,0.00856,0.009695,0.002808,0.003167,0.0,0.012032,0.00631,1,0.00682,0.015139,0.001612,0.009155,0.001558,0.001757,0.013816,0.006913,0.0
10,0.017379,0.019552,0.004342,0.0,0.002128,0.0,0.009238,0.002119,0.00682,1,0.01017,0.001083,0.009841,0.004188,0.003541,0.010442,0.004644,0.00418


In [328]:
data_neighbours = pd.DataFrame(index=data_ibs.columns, columns=range(1, 7))

# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0, len(data_ibs.columns)):
    data_neighbours.iloc[i, :6] = data_ibs.iloc[:, i].sort_values(ascending=False)[:6].index

In [329]:
data_neighbours.drop(columns=1, inplace=True)

In [330]:
data_neighbours

,2,3,4,5,6
game_id,,,,,
1,11,10,2,14,9
2,11,10,7,16,1
3,17,11,16,7,9
4,11,5,17,7,3
5,4,3,17,9,13
6,13,3,16,11,1
7,16,11,2,13,9
8,11,9,1,3,13
9,11,16,7,1,3


In [331]:
data_neighbours.index.name = 'game_id'

In [341]:
data_neighbours.rename(columns={data_neighbours.columns[0]: 'sim1'}, inplace=True)
data_neighbours.rename(columns={data_neighbours.columns[1]: 'sim2'}, inplace=True)
data_neighbours.rename(columns={data_neighbours.columns[2]: 'sim3'}, inplace=True)
data_neighbours.rename(columns={data_neighbours.columns[3]: 'sim4'}, inplace=True)
data_neighbours.rename(columns={data_neighbours.columns[4]: 'sim5'}, inplace=True)

In [342]:
data_neighbours

,sim1,sim2,sim3,sim4,sim5
game_id,,,,,
1,11,10,2,14,9
2,11,10,7,16,1
3,17,11,16,7,9
4,11,5,17,7,3
5,4,3,17,9,13
6,13,3,16,11,1
7,16,11,2,13,9
8,11,9,1,3,13
9,11,16,7,1,3


In [350]:
import sqlite3

conn = sqlite3.connect('../database/gamender.db')
c = conn.cursor()
c.execute(
    'CREATE TABLE IF NOT EXISTS collaboration (game_id number, sim1 number,sim2 number,sim3 number, sim4 number,sim5 number)')
conn.commit()


In [352]:
data_neighbours.to_sql('collaboration', conn, if_exists='replace', index=True)

In [404]:
listofgame

In [405]:
len(listofgame)

8

In [ ]:
def remove_alrdy_played(a, b):
    for i in a[:]:
        if i in b:
            a.remove(i)

In [406]:
from collections import Counter
from itertools import chain


# takes list of user liked gamed (l) and colab data
def gettopgame(l, data):
    common = []
    listoflists = []
    if len(l) == 1:
        common.append(data.iloc[l[0] - 1, 0])
        return common
    else:
        n = len(l)
        for j in range(0, n):
            listoflists.append(list(data.iloc[l[j] - 1]))
        common = list(chain.from_iterable(listoflists))
        commonofmany = [k for k, v in Counter(common).items() if v > 1]
        if len(commonofmany) > 0:
            remove_alrdy_played(commonofmany, l)
            return commonofmany
        else:
            remove_alrdy_played(common, l)
            return list(dict.fromkeys(common))

In [407]:
common = gettopgame(listofgame, data_neighbours)

In [411]:
common

[11, 14, 13, 7, 3]